In [2]:
from pdfminer.pdfdocument import PDFDocument, PDFNoOutlines
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTPage, LTChar, LTAnno, LAParams, LTTextBox, LTTextLine
import numpy as np
import pandas as pd
from pprint import pprint
from pdfminer.pdfpage import PDFPage
from decimal import Decimal


class PDFPageDetailedAggregator(PDFPageAggregator):
    def __init__(self, rsrcmgr, pageno=1, laparams=None):
        PDFPageAggregator.__init__(self, rsrcmgr, pageno=pageno, laparams=laparams)
        self.rows = []
        self.page_number = 0
    def receive_layout(self, ltpage):
        def render(item, page_number):
            if isinstance(item, LTPage) or isinstance(item, LTTextBox):
                for child in item:
                    render(child, page_number)
            elif isinstance(item, LTTextLine):
                child_str = ''
                for child in item:
                    if isinstance(child, (LTChar, LTAnno)):
                        child_str += child.get_text()
                child_str = ' '.join(child_str.split()).strip()
                if child_str:
                    row = [page_number, item.bbox[0], item.bbox[1], item.bbox[2], item.bbox[3], child_str] # bbox == (x1, y1, x2, y2)
                    self.rows.append(row)
                for child in item:
                    render(child, page_number)
            return
        render(ltpage, self.page_number)
        self.page_number += 1
        self.rows = sorted(self.rows, key = lambda x: (x[0], -x[2]))
        self.result = ltpage



fp = open('./profile.pdf', 'rb')
parser = PDFParser(fp)
doc = PDFDocument(parser)

rsrcmgr = PDFResourceManager()
laparams = LAParams()
device = PDFPageDetailedAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)

for page in PDFPage.create_pages(doc):
    interpreter.process_page(page)
    # receive the LTPage object for this page
    device.get_result()

my_array = device.rows

In [2]:
df = pd.DataFrame.from_records(my_array)
df.columns = ['pn','x0','y0','x1','y1','line']
df['y_dif'] = round(df['y1']-df['y0'])

grouped_y0 = df.groupby(['pn'])['y0'].max().reset_index().sort_values('pn', ascending=True)
grouped_y1 = df.groupby(['pn'])['y1'].max().reset_index().sort_values('pn', ascending=True)

grouped_y0['y0_'] = grouped_y0['y0'].shift(-1)
grouped_y1['y1_'] = grouped_y1['y1'].shift(-1)

grouped_y0.fillna(0, inplace=True)
grouped_y1.fillna(0, inplace=True)

grouped_y0['cum_y0'] = grouped_y0.loc[::-1, 'y0_'].cumsum()[::-1]
grouped_y1['cum_y1'] = grouped_y1.loc[::-1, 'y1_'].cumsum()[::-1]

grouped_y0 = grouped_y0[['pn','cum_y0']]
grouped_y1 = grouped_y1[['pn','cum_y1']]

df = df.merge(grouped_y0, on='pn')
df = df.merge(grouped_y1, on='pn')

df['y0'] = df['y0'] + df['cum_y0']
df['y1'] = df['y1'] + df['cum_y1']

df.drop(['cum_y0', 'cum_y1'], axis=1, inplace=True)

In [3]:
indent_cat = {}
pct = 0.001
for index, row in df.iterrows():
    feature = row['x0']
    bin_found = False
    for bin_key,bin_arr in indent_cat.items():
        if feature >= (bin_key * (1.0-pct)) and (bin_key * (1.0+pct)) >= feature:
            bin_found = True
            bin_arr.append(index)
    if not bin_found:
        indent_cat[feature] = [index]
        
def labeling(x,cat_):
    for key, val in cat_.items():
        if  x in val:
            return list(sorted(cat_)).index(key)


df['indent_label'] = df.index.map(lambda x: labeling(x, indent_cat))

In [4]:
df_1 = df[df['indent_label'] == 1].copy()
df_0 = df[df['indent_label'] == 0].copy()

In [5]:
arr = []
def title(row):
    if row.y_dif >= 22:
        arr.append(row.line)
    
    return arr[-1]

df_1['Title'] = df_1.apply(lambda row: title(row), axis = 1)

In [ ]:
df_1

In [ ]:
df_ex = df_1[df_1['Title'] == 'Experience'].copy()
df_ed = df_1[df_1['Title'] == 'Education'].copy()

In [7]:
import re
pattern = r'([a-zA-Z]+\s\d{4})\s-\s(?:([a-zA-Z]+\s\d{4})|(\w+))'
regex = re.compile(pattern, re.IGNORECASE)


i = - 1

arr = []

for index, row in df_ex.iterrows():
    
    if row['y_dif'] == 17:
        
        arr.append({})
        arr[-1]['company'] = row['line']
        
    elif row['y_dif'] == 16:
        if 'experience' not in arr[-1]:
            arr[-1]['experience'] =  []
            
        arr[-1]['experience'].append({})
        arr[-1]['experience'][-1]['position'] = row['line']
        
    elif row['y_dif'] == 15 and regex.findall(row['line']):
        if 'experience' in arr[-1]:
            t = arr[-1]['experience'][-1]['date_period'] if('date_period' in arr[-1]['experience'][-1]) else []
            arr[-1]['experience'][-1]['date_period'] = t + regex.findall(row['line'])
    
    elif row['y_dif'] == 15:
        if 'experience' in arr[-1]:
            t = arr[-1]['experience'][-1]['meta'] if('meta' in arr[-1]['experience'][-1]) else ''
            arr[-1]['experience'][-1]['meta'] = t + ' ' + row['line']
        

In [8]:
arr

[{'company': 'Insharp Technologies (Private)Limited',
  'experience': [{'position': 'Software Engineer',
    'date_period': [('April 2019', '', 'Present')],
    'meta': ' Sri Lanka .Net MVC Entity web application development. Java Android objective C IOS mobile application development Front end development with Reactjs and Vuejs agile project coordinate'}]},
 {'company': 'Fiverr',
  'experience': [{'position': 'Consultant',
    'date_period': [('June 2018', '', 'Present')],
    'meta': ' Sri Lanka consulting and adviceing enterprise level application development requirments. design system architecture of multi level application and utilities manage strong professional software development team.'},
   {'position': 'Full-stack Developer',
    'date_period': [('August 2017', '', 'Present')],
    'meta': ' Sri Lanka design and develop customizable mobile and web application with specialized in self-service KIOSK and Mobile POS application'},
   {'position': 'Software Developer',
    'date_

In [10]:
i = - 1

arr_ed = []

for index, row in df_ed.iterrows():
    
    if row['y_dif'] == 17:
        
        arr_ed.append({})
        arr_ed[-1]['institute'] = row['line']
        
    elif row['y_dif'] == 15:
        t = arr_ed[-1]['degree'] if ('degree' in arr_ed[-1]) else ''
        arr_ed[-1]['degree']= t + ' '+ row['line']

In [11]:
arr_ed

[{'institute': 'BCS, The Chartered Institute for IT',
  'degree': " Bachelor's degree, Information Technology · (2015 - 2018)"},
 {'institute': 'institute of java technology Sri lanka(ijts)',
  'degree': ' Bachelor of Science (B.Sc.), Computer Software Engineering · (2014 - 2017)'},
 {'institute': 'Badulla Central Collage',
  'degree': ' Business/Commerce, General · (2005 - 2013)'}]

In [13]:
arr = []
def title(row):
    if row.y_dif >= 18:
        arr.append(row.line)
    return arr[-1]

df_0['Title'] = df_0.apply(lambda row: title(row), axis = 1)

In [14]:
df_0

,pn,x0,y0,x1,y1,line,y_dif,indent_label,Title
0,0,21.6,1468.194500,66.372000,1502.666000,Contact,18.0,0,Contact
2,0,21.6,1449.565000,120.804001,1480.519000,0719977365 (Mobile),15.0,0,Contact
3,0,21.6,1436.965002,155.947500,1467.919002,nazik.naweed24@gmail.com,15.0,0,Contact
5,0,21.6,1412.411502,153.578000,1444.069002,www.linkedin.com/in/nazik-,15.0,0,Contact
6,0,21.6,1398.011500,164.654999,1429.669000,naweed-45928ab8 (LinkedIn),15.0,0,Contact
9,0,21.6,1362.094497,77.903000,1396.565997,Top Skills,18.0,0,Top Skills
11,0,21.6,1343.464994,43.776000,1374.418994,Java,15.0,0,Top Skills
13,0,21.6,1325.865003,70.593000,1356.819003,JavaScript,15.0,0,Top Skills
15,0,21.6,1308.264997,57.762000,1339.218997,JavaSE,15.0,0,Top Skills
18,0,21.6,1272.694496,85.924000,1307.165996,Languages,18.0,0,Languages


In [ ]:
df_co = df_0[df_0['Title'] == 'Contact'].copy()
df_sk = df_0[df_0['Title'] == 'Top Skills'].copy()
df_la = df_0[df_0['Title'] == 'Languages'].copy()
df_cert = df_0[df_0['Title'] == 'Certifications'].copy()

In [17]:
i = - 1

arr_co = []

for index, row in df_co.iterrows():
    
    if row['y_dif'] == 18:
        
        arr_co.append({})
        
    if row['y_dif'] == 15:
        t = arr_co[-1]['contact'] if ('contact' in arr_co[-1]) else []
        arr_co[-1]['contact']=  t + [row['line']]

In [18]:
i = - 1

arr_sk = []

for index, row in df_sk.iterrows():
    
    if row['y_dif'] == 18:
        
        arr_sk.append({})
        
    if row['y_dif'] == 15:
        t = arr_sk[-1]['skills'] if ('skills' in arr_sk[-1]) else []
        arr_sk[-1]['skills']=  t + [row['line']]

In [19]:
i = - 1

arr_la = []

for index, row in df_la.iterrows():
    
    if row['y_dif'] == 18:
        
        arr_la.append({})
        
    if row['y_dif'] == 15:
        t = arr_la[-1]['languages'] if ('languages' in arr_la[-1]) else []
        arr_la[-1]['languages']=  t + [row['line']]

In [44]:
i = - 1

arr_cert = []

for index, row in df_cert.iterrows():
    
    if row['y_dif'] == 18:
        
        arr_cert.append({})
        
    if row['y_dif'] == 15:
        t = arr_cert[-1]['certifications'] if ('certifications' in arr_cert[-1]) else []
        arr_cert[-1]['certifications'] = t + [row['line']]
        

In [45]:
arr_cert

[{'certifications': ['java script tutorial course',
   'BCS Level 4 Certificate in IT',
   'SQL tutorial course',
   'Diploma in IT & E Commerce',
   '(international)']}]